# Stochastic Bernoulli Bandit (Multi-Objective with Cost-Control)

In [1]:
import numpy as np
from rich import print

from pybandits.model import Beta, BetaMOCC
from pybandits.smab import SmabBernoulliMOCC, create_smab_bernoulli_mo_cc_cold_start

In [2]:
# print 2 decimal places in the notebook
%precision %.2f

'%.2f'

## 1. Initialization
The following two options are available to initialize the bandit.

### 1.1 Initialize via class constructor

You can initialize the bandit via the class constructor `SmabBernoulliMOCC()`. This is useful to impute prior knowledge on the Beta distributions.

In [3]:
n_objectives = 2

mab = SmabBernoulliMOCC(
    actions={
        "a1": BetaMOCC(counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)], cost=30),
        "a2": BetaMOCC(counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)], cost=10),
        "a3": BetaMOCC(counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)], cost=20),
    }
)

In [4]:
print(mab)

SmabBernoulliMOCC(
    actions={
        'a1': BetaMOCC(
            counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)],
            cost=30.0
        ),
        'a2': BetaMOCC(
            counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)],
            cost=10.0
        ),
        'a3': BetaMOCC(
            counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)],
            cost=20.0
        )
    },
    strategy=MultiObjectiveCostControlBandit()
)

### 1.2 Initialize via utility function (for cold start)

You can initialize the bandit via the utility function `create_smab_bernoulli_mo_cc_cold_start()`. This is particulary useful in a cold start setting when there is no prior knowledge on the Beta distruibutions. In this case for all Betas `n_successes` and `n_failures` are set to `1`.

In [5]:
# list of action IDs with their cost
action_ids_cost = {"a1": 30, "a2": 10, "a3": 20}
n_objectives = 2

# generate a smab bernoulli in cold start settings
mab = create_smab_bernoulli_mo_cc_cold_start(action_ids_cost=action_ids_cost, n_objectives=n_objectives)

In [6]:
print(mab)

SmabBernoulliMOCC(
    actions={
        'a1': BetaMOCC(
            counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)],
            cost=30.0
        ),
        'a2': BetaMOCC(
            counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)],
            cost=10.0
        ),
        'a3': BetaMOCC(
            counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)],
            cost=20.0
        )
    },
    strategy=MultiObjectiveCostControlBandit()
)

## 2. Function `predict()`

In [7]:
help(mab.predict)

Help on method predict in module pybandits.smab:

predict(n_samples: pydantic.types.PositiveInt = 1, forbidden_actions: Optional[Set[pybandits.base.ActionId]] = None) -> Tuple[List[pybandits.base.ActionId], List[Dict[pybandits.base.ActionId, pybandits.base.Probability]]] method of pybandits.smab.SmabBernoulliMOCC instance
    Predict actions.
    
    Parameters
    ----------
    n_samples : int > 0, default=1
        Number of samples to predict.
    forbidden_actions : Optional[Set[ActionId]], default=None
        Set of forbidden actions. If specified, the model will discard the forbidden_actions and it will only
        consider the remaining allowed_actions. By default, the model considers all actions as allowed_actions.
        Note that: actions = allowed_actions U forbidden_actions.
    
    Returns
    -------
    actions: List[ActionId] of shape (n_samples,)
        The actions selected by the multi-armed bandit model.
    probs: List[Dict[ActionId, Probability]] of shape (n

In [8]:
# predict for 5 samples
actions, probs = mab.predict(n_samples=5)

In [9]:
actions

['a3', 'a3', 'a2', 'a3', 'a2']

In [10]:
probs

[{'a1': [0.75, 0.55], 'a2': [0.78, 0.29], 'a3': [0.79, 0.83]},
 {'a1': [0.95, 0.28], 'a2': [0.22, 0.23], 'a3': [0.99, 0.95]},
 {'a1': [0.22, 0.64], 'a2': [0.62, 0.50], 'a3': [0.30, 0.12]},
 {'a1': [0.19, 0.79], 'a2': [0.02, 0.70], 'a3': [0.27, 0.72]},
 {'a1': [0.38, 0.03], 'a2': [0.80, 0.55], 'a3': [0.79, 0.04]}]

In [11]:
# predict for 5 samples with forbidden actions, in this case `a1` will never be predicted.
actions, probs = mab.predict(n_samples=5, forbidden_actions=["a1"])

In [12]:
actions

['a3', 'a2', 'a2', 'a2', 'a2']

In [13]:
probs

[{'a2': [0.12, 0.45], 'a3': [0.38, 0.90]},
 {'a2': [0.10, 0.96], 'a3': [0.58, 0.20]},
 {'a2': [0.92, 0.85], 'a3': [0.31, 0.65]},
 {'a2': [0.60, 0.04], 'a3': [0.45, 0.97]},
 {'a2': [0.87, 0.51], 'a3': [0.74, 0.35]}]

## 3. Function `update()`

In [14]:
help(mab.update)

Help on method update in module pybandits.smab:

update(actions: List[pybandits.base.ActionId], rewards: List[List[pybandits.base.BinaryReward]]) method of pybandits.smab.SmabBernoulliMOCC instance
    Update the stochastic Bernoulli bandit given the list of selected actions and their corresponding binary
    rewards.
    
    Parameters
    ----------
    actions : List[ActionId] of shape (n_samples,), e.g. ['a1', 'a2', 'a3', 'a4', 'a5']
        The selected action for each sample.
    rewards : List[Union[BinaryReward, List[BinaryReward]]] of shape (n_samples, n_objectives)
        The binary reward for each sample.
            If strategy is not MultiObjectiveBandit, rewards should be a list, e.g.
                rewards = [1, 0, 1, 1, 1, ...]
            If strategy is MultiObjectiveBandit, rewards should be a list of list, e.g. (with n_objectives=2):
                rewards = [[1, 1], [1, 0], [1, 1], [1, 0], [1, 1], ...]



In [15]:
# simulate rewards from the environment
rewards = [[1, 1], [1, 0], [1, 1], [1, 0], [1, 1]]

In [16]:
# update
mab.update(actions=actions, rewards=rewards)
print(mab)

SmabBernoulliMOCC(
    actions={
        'a1': BetaMOCC(
            counters=[Beta(n_successes=1, n_failures=1), Beta(n_successes=1, n_failures=1)],
            cost=30.0
        ),
        'a2': BetaMOCC(
            counters=[Beta(n_successes=7, n_failures=3), Beta(n_successes=7, n_failures=3)],
            cost=10.0
        ),
        'a3': BetaMOCC(
            counters=[Beta(n_successes=3, n_failures=1), Beta(n_successes=3, n_failures=1)],
            cost=20.0
        )
    },
    strategy=MultiObjectiveCostControlBandit()
)

## 4. Example of usage

Simulate 10 updates, for each updates we predict actions for a batch of 1000 samples and then we update the bandit given the rewards.

In [17]:
n_updates = 10
batch_size = 1000

for _ in range(n_updates):
    # predict
    actions, _ = mab.predict(n_samples=batch_size)

    # simulate rewards from the environment
    rewards = np.reshape(
        a=[np.random.choice([0, 1], size=batch_size), np.random.choice([0, 1], size=batch_size)],
        newshape=(batch_size, 2),
    ).tolist()

    # update
    mab.update(actions=actions, rewards=rewards)

In [18]:
print(mab)

SmabBernoulliMOCC(
    actions={
        'a1': BetaMOCC(
            counters=[Beta(n_successes=450, n_failures=488), Beta(n_successes=450, n_failures=488)],
            cost=30.0
        ),
        'a2': BetaMOCC(
            counters=[Beta(n_successes=8541, n_failures=8325), Beta(n_successes=8541, n_failures=8325)],
            cost=10.0
        ),
        'a3': BetaMOCC(
            counters=[Beta(n_successes=1110, n_failures=1102), Beta(n_successes=1110, n_failures=1102)],
            cost=20.0
        )
    },
    strategy=MultiObjectiveCostControlBandit()
)